In [8]:
import pandas as pd

import geopandas as gpd
import matplotlib.pyplot as plt

import os

In [9]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [10]:
# RUTA ONEDRIVE
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [11]:
path_shp_recorridos = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\RECORRIDOS.shp'
path_shp_plan = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\PLANIFICACION.shp'
path_xlsx_parte = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\parte_diario_pulverizacion.xlsx'

In [12]:
# Cargar los archivos Shapefile
gdf_recorridos = gpd.read_file(path_shp_recorridos)
gdf_plan = gpd.read_file(path_shp_plan)

In [13]:
idd = 24

In [14]:
selec_recorridos = gdf_recorridos[gdf_recorridos['idd']==idd]

In [15]:
selec_plan = gdf_plan[gdf_plan['idd'] == idd]

In [16]:
fig, ax = plt.subplots(figsize=(10, 8))
# Graficar las capas
selec_recorridos.plot(ax=ax, color="blue", alpha=0.5, edgecolor="black", label="Recorridos")
selec_plan.plot(ax=ax, color="red", alpha=0.5, edgecolor="black", label="Planificación")

# Configurar la visualización
ax.set_title("Mapa de Recorridos y Planificación")
ax.legend()

# Mostrar el gráfico
plt.show()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_22916\2512288014.py:8: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  ax.legend()


In [17]:
selec_recorridos["fecha"] = pd.to_datetime(selec_recorridos["fecha"])

C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [18]:
# Obtener la fecha mínima y máxima
fecha_min = selec_recorridos["fecha"].min()
fecha_max = selec_recorridos["fecha"].max()
fecha_intermedia = fecha_min + (fecha_max - fecha_min) / 2

In [19]:
# Convertir las fechas al formato dd/mm/aaaa
fecha_min_str = fecha_min.strftime("%d/%m/%Y")
fecha_max_str = fecha_max.strftime("%d/%m/%Y")
fecha_intermedia_str = fecha_intermedia.strftime("%d/%m/%Y")
# Obtener el número de semana de la fecha intermedia
numero_semana = fecha_intermedia.isocalendar().week

In [20]:
area_total_pulv = selec_recorridos['area'].sum()
area_total_pulv

91.95400045976999

In [21]:
num_vuelos = len(selec_recorridos)
num_vuelos

27

In [22]:
selec_recorridos.head(3)

file      fecha      hora           id  \
478  T50-1_20250328064003_R4903070624.kml 2025-03-28  06:40:03  R4903070624   
479  T50-1_20250328065244_R5103296094.kml 2025-03-28  06:52:44  R5103296094   
480  T50-1_20250328070425_R5283499017.kml 2025-03-28  07:04:25  R5283499017   

     drone               ctrl_id          pilot fl_time  mode  height  \
478  T50-1  1581F6BUB24630011R08  MARIO SANCHEZ   10:07  Auto     3.5   
479  T50-1  1581F6BUB24630011R08  MARIO SANCHEZ    9:21  Auto     3.5   
480  T50-1  1581F6BUB24630011R08  MARIO SANCHEZ   10:25  Auto     3.5   

     spacing   fl_speed      area   spray  idd  \
478     10.0  29.880001  3.761333  37.729   24   
479     10.0  29.880001  3.514667  35.266   24   
480     10.0  29.880001  3.550667  35.612   24   

                                              geometry  
478  LINESTRING (480552.498 8088828.069, 480557.273...  
479  LINESTRING (480552.650 8088828.022, 480557.881...  
480  LINESTRING (480552.424 8088829.989, 480556.986...

In [23]:
# Convert "fl_time" to total seconds
selec_recorridos["fl_time_sec"] = selec_recorridos["fl_time"].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
# Sum all seconds
total_seconds = selec_recorridos["fl_time_sec"].sum()
# Convert back to hh:mm:ss format
hours = total_seconds // 3600
minutes = (total_seconds % 3600) // 60
seconds = total_seconds % 60
total_flight_time = f"{hours:02}:{minutes:02}:{seconds:02}"
total_flight_time

C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


'04:27:14'

In [24]:
total_caudal = selec_recorridos['spray'].sum()
total_caudal

924.6149999999999

In [25]:
df_parte = pd.read_excel(path_xlsx_parte)

In [26]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA DE INICIO'] = pd.to_datetime(df_parte['HORA DE INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [27]:
df_parte.head(3)

Nº DE LABOR      FECHA  SEMANA  INSTITUCION  COD CANERO  \
0   PA-2025-1 2025-03-07    10.0           50       41594   
1   PA-2025-1 2025-03-07    10.0           50       41594   
2   PA-2025-1 2025-03-07    10.0           50       41594   

                     NOMBRE CANERO  COD PROPIEDAD NOMBRE PROPIEDAD    LOTE  \
0  AGROPECUARIA CAMPO DULCE S.R.L.             30      CAMPO DULCE  EP-L31   
1  AGROPECUARIA CAMPO DULCE S.R.L.             30      CAMPO DULCE  EP-L16   
2  AGROPECUARIA CAMPO DULCE S.R.L.             30      CAMPO DULCE   ER-L6   

    HECTAREA  ...  OS TEMP MAX TEMP MIN    TIPO CULTIVO  TIPO DESARROLLO  \
0  30.260942  ...   1     25.0     20.0  CAÑA DE AZUCAR       MADURACION   
1  23.216246  ...   1     25.0     20.0  CAÑA DE AZUCAR       MADURACION   
2  19.683505  ...   1     25.0     20.0  CAÑA DE AZUCAR       MADURACION   

   NUM VUELOS  HORAS VUELO  TOTAL CAUDAL  idd  AREA PULVERIZADO  
0       104.0     15:03:42      2952.664  1.0         28.333271  
1       104.0     15:03:42      2952.664  1.0         21.634272  
2       104.0     15:03:42      2952.664  1.0         19.160233  

[3 rows x 35 columns]

In [28]:
# extraer primer dato de temp max
temp_max = df_parte[df_parte['idd'] == idd]['TEMP MAX'].iloc[0]
temp_min = df_parte[df_parte['idd'] == idd]['TEMP MIN'].iloc[0]

In [29]:
temp = round((temp_max + temp_min) / 2)
temp

28

In [30]:
# Aplicar el buffer a la geometría
selec_recorridos["geometry"] = selec_recorridos.geometry.buffer(5.5)

C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [31]:
# Disolver todas las geometrías en un solo polígono
buffer_dissolved = selec_recorridos.dissolve()

In [32]:
buffer_dissolved

geometry  \
0  MULTIPOLYGON (((480477.457 8089400.584, 480477...   

                                   file      fecha      hora           id  \
0  T50-1_20250328064003_R4903070624.kml 2025-03-28  06:40:03  R4903070624   

   drone               ctrl_id          pilot fl_time  mode  height  spacing  \
0  T50-1  1581F6BUB24630011R08  MARIO SANCHEZ   10:07  Auto     3.5     10.0   

    fl_speed      area   spray  idd  fl_time_sec  
0  29.880001  3.761333  37.729   24          607

In [33]:
# Calcular la intersección
interseccion = gpd.overlay(buffer_dissolved, selec_plan, how="intersection")

In [34]:
# Calcular el área de cada polígono de la intersección
interseccion["area_rociado"] = interseccion.geometry.area / 10000

In [35]:
resumen = interseccion[['id_2', 'unidad_05', 'area_2', 'area_rociado']].copy()

In [36]:
suma_area_rociada = resumen['area_rociado'].sum()
area_rociada_real = area_total_pulv  # aquí pon tu valor real
# Factor inicial de ajuste
factor_ajuste = area_rociada_real / suma_area_rociada
# Ajuste proporcional inicial
resumen['area_ajustada'] = resumen['area_rociado'] * factor_ajuste
# Verificar que no exceda area_2 y corregir iterativamente si es necesario
while any(resumen['area_ajustada'] > resumen['area_2']):
    exceso = resumen['area_ajustada'] > resumen['area_2']
    resumen.loc[exceso, 'area_ajustada'] = resumen.loc[exceso, 'area_2']
    # Recalcular factor solo con las filas no ajustadas al máximo permitido
    area_ajustada_parcial = resumen.loc[~exceso, 'area_ajustada'].sum()
    area_restante = area_rociada_real - resumen.loc[exceso, 'area_ajustada'].sum()
    # Evitar división por cero
    if area_ajustada_parcial == 0:
        break
    nuevo_factor = area_restante / area_ajustada_parcial
    resumen.loc[~exceso, 'area_ajustada'] *= nuevo_factor

In [37]:
resumen["porcen_rociado"] = (resumen["area_ajustada"]) / (resumen["area_2"])

In [38]:
resumen["dif_area"] = resumen["area_2"] - resumen["area_ajustada"]

In [39]:
for i, row in resumen.iterrows():
    df_parte.loc[(df_parte['ID'] == row['id_2']), 'AREA PULVERIZADO'] = row['area_ajustada']

In [40]:
df_parte.loc[(df_parte['idd'] == idd), 'FECHA'] = fecha_intermedia
df_parte.loc[(df_parte['idd'] == idd), 'SEMANA'] = numero_semana
df_parte.loc[(df_parte['idd'] == idd), 'HORA DE INICIO'] = fecha_min
df_parte.loc[(df_parte['idd'] == idd), 'HORA FINAL'] = fecha_max
df_parte.loc[(df_parte['idd'] == idd), 'TEMPERATURA'] = temp
df_parte.loc[(df_parte['idd'] == idd), 'NUM VUELOS'] = num_vuelos
df_parte.loc[(df_parte['idd'] == idd), 'HORAS VUELO'] = total_flight_time
df_parte.loc[(df_parte['idd'] == idd), 'TOTAL CAUDAL'] = total_caudal

In [41]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

In [42]:
# Calcular el total de area_2
total_area_2 = resumen["area_2"].sum()
total_area_rociado = resumen["area_rociado"].sum()
# Crear una nueva columna con el porcentaje
resumen["porcen_rociado"] = (resumen["area_rociado"] / total_area_rociado)

In [43]:
diferencia = area_total_pulv - total_area_rociado 
diferencia

1.2892873639010674

In [44]:
resumen["area_diff"] = resumen["porcen_rociado"] * diferencia

In [45]:
resumen["area_rociado_ajustado"] = resumen["area_rociado"] + resumen["area_diff"]

In [46]:
resumen = resumen[['id_2', 'unidad_05', 'area_2', 'area_rociado_ajustado']].copy()

In [47]:
total_area_rociado_ajustado = resumen["area_rociado_ajustado"].sum()
resumen["porcen_rociado"] = (resumen["area_rociado_ajustado"]) / (resumen["area_2"])

In [48]:
resumen["dif_area"] = resumen["area_2"] - resumen["area_rociado_ajustado"]

In [49]:
for i, row in resumen.iterrows():
    df_parte.loc[(df_parte['ID'] == row['id_2']), 'AREA PULVERIZADO'] = row['area_rociado_ajustado']

In [50]:
df_parte.loc[(df_parte['idd'] == idd), 'FECHA'] = fecha_intermedia
df_parte.loc[(df_parte['idd'] == idd), 'SEMANA'] = numero_semana
df_parte.loc[(df_parte['idd'] == idd), 'HORA DE INICIO'] = fecha_min
df_parte.loc[(df_parte['idd'] == idd), 'HORA FINAL'] = fecha_max
df_parte.loc[(df_parte['idd'] == idd), 'TEMPERATURA'] = temp
df_parte.loc[(df_parte['idd'] == idd), 'NUM VUELOS'] = num_vuelos
df_parte.loc[(df_parte['idd'] == idd), 'HORAS VUELO'] = total_flight_time
df_parte.loc[(df_parte['idd'] == idd), 'TOTAL CAUDAL'] = total_caudal

In [51]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)